This notebook is a part of the work 'Nucleation patterns of polymer crystals analyzed by machine learning models' and is written by Atmika Bhardwaj (bhardwaj@ipfdd.de).

The following machine learning (ML) models are employed: AutoEncoders (AE), Hierarchical Clustering (HC), Gaussian Mixture Models (GMM), Multilayer Perceptron (MLP) to harness the capabilities of ML algorithms to unfold the details associated with the different phases that emerge during polymer crystallization. We start by reading a coordinate file (dump file from LAMMPS) and analyze it to study the local environmental information of every coarse-grained bead. Then, we train an AE to compress that information and train a GMM on the compressed data to classify each coarse-grained into amorphous or crytalline depending on its enviromental fingerprint.

This notebook imports all the functions from another file called all_functions.py. Another file control_parameters.py imports all the tuned parameters.



In [ ]:
import glob
import random
import pickle
import keras
import numpy                  as     np
import pandas                 as     pd
from   numpy.random           import seed
import matplotlib.pyplot      as     plt
import matplotlib.cm          as     cm
import h5py                   as     h5py
from   itertools              import product, combinations
from   mpl_toolkits.mplot3d   import Axes3D
from   natsort                import natsorted
from   sklearn                import mixture
from   keras.models           import load_model
from   all_functions          import *
import matplotlib.ticker      as     mticker
import our_colors
import control_parameters

ocol = our_colors.generate_colors()
olab = our_colors.generate_labels()

param  = control_parameters.get_defaults()
F10min = param["F10min"]
F10max = param["F10max"]
fig10_ticks = np.arange(F10min+1, F10max+1,2).astype(int)

seed(1)
plt.rcParams["figure.figsize"]  = (6,5)
plt.rcParams["figure.dpi"]      = 500
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

plt.rcParams.update({"text.usetex": False})

ps_am       = 1.0               # crystalline point size
class_scale = 3.0
ps_cr       = class_scale*ps_am # amorphous point size

skip_training   = True
skip_predicting = True
skip_SL         = True


## AE training

In [ ]:
hid_dim          = 14
depth            = 14
nEpochs          = 600
VL               = 1
batchSize        = 20
validationSplit  = 0.2
L2               = 10e-7
LR               = 10e-5
g                = natsorted(glob.glob('../../01_raw_data/dump_files/dump.*'))


In [ ]:
h5_file          = h5py.File('../all_fingerprints.h5', 'r')
h5_keys          = natsorted(list(h5_file.keys()))
file_num         = -1
obj              = ReadCoordinates(g[file_num], param["chainlength"])
rnd              = get_slice_indices(obj, 100, 15, axis = 2)

if not skip_training: 
    np.savetxt('../../02_processed_data/Training_data/ae_training_idxs.csv', rnd, delimiter=",")

ae_training_data = 0    

if (skip_training):
    ae_training_data = np.loadtxt('../../02_processed_data/Training_data/ae_training_data.csv',delimiter=",")
else:
    ae_training_data = np.array(h5_file.get(h5_keys[file_num]))[rnd]
    ae_training_data = np.delete(ae_training_data, 10, axis = 1)
    np.savetxt('../../02_processed_data/Training_data/ae_training_data.csv', ae_training_data, delimiter=",")
    
print(ae_training_data.shape)


In [ ]:
encoded = 0

if (skip_training):
    encoded = np.loadtxt('../../02_processed_data/Training_data/ae_training_encoded.csv', delimiter=",")
else:
    autoencoder, encoder, hist, encoded = fixedSeeds(2, hid_dim, ae_training_data, depth, "elu", L2, VL, nEpochs, LR, validationSplit, batchSize, '../Models/ae_model.sav')
    np.savetxt('../../02_processed_data/Training_data/ae_training_encoded.csv', encoded, delimiter=",")
    histories=[hist]
    l  = hist.history["loss"]
    lv = hist.history["val_loss"]
    t  = np.arange(len(l))
    np.savetxt('../../03_figures/FigS1/FigS1.csv', [lv, l], delimiter=",")
print(encoded.shape)


In [ ]:
#if not skip_training:
filename = '../../03_figures/FigS1/FigS1.csv'
data    = np.loadtxt(filename, delimiter=",")

fig = plt.figure()
plt.axes(xscale='log', yscale='log')
plt.grid(alpha = 0.3)
lv = data[0]
l  = data[1]
t  = np.arange(len(l))
plt.plot(t, l,  label=olab["sAE"]+" training loss")
plt.plot(t, lv, label=olab["sAE"]+" validation loss")

plt.ylim(0.022, 0.2)    
plt.xlabel("Epochs",  fontsize = 20)
plt.ylabel("Loss (MSE)",  fontsize = 20)
lgnd     = plt.legend(fontsize = 20)
plt.tight_layout()
plt.savefig('../../03_figures/FigS1/FigS1.png')



## AE labels with GMM training

In [ ]:
filename = '../../02_processed_data/Training_data/ae_training_encoded.csv'
encoded  = np.loadtxt(filename, delimiter=",")

np.savetxt('../../03_figures/Fig06/Fig06.csv', encoded, delimiter=",")
filename = '../../03_figures/Fig06/Fig06.csv'
encoded  = np.loadtxt(filename, delimiter=",")

gmm      = 0
filename = '../../02_processed_data/Models/GMM_model.sav'

if skip_training:
    gmm  = pickle.load(open(filename, 'rb'))
else:
    gmm  = mixture.GaussianMixture(n_components = 2, covariance_type = "full")
    pickle.dump(gmm, open(filename, 'wb'))

yhc = gmm.fit_predict(encoded)

fig = plt.figure()
p1  = encoded[yhc==1]
plt.scatter(p1[:,1], p1[:,0], c = ocol["amorph"],  s = ps_am, label = olab["sAE"]+olab["am"])
p1  = encoded[yhc==0]
plt.scatter(p1[:,1], p1[:,0], c = ocol["AE"],      s = ps_cr, label = olab["sAE"]+olab["cr"])

our_colors.formatMap(plt)

plt.savefig('../../03_figures/Fig06/Fig06a.png',bbox_inches="tight")


## Predictions on the whole time series using the trained model

In [ ]:
if not skip_predicting:
    all_labels_ae   = []
    all_encoded     = []
    autoenc         = load_model('../../02_processed_data/Models/ae_model.sav')
    gmm             = pickle.load(open('../../02_processed_data/Models/GMM_model.sav', 'rb'))

    for i in np.arange(len(g)):
        training_data  = np.array(h5_file.get(h5_keys[i]))
        training_data  = np.delete(training_data, 10, axis = 1)
        encoded_output = autoenc.predict(training_data)
        all_encoded.append(encoded_output)  

        label_ae       = gmm.predict(encoded_output)
        all_labels_ae.append(label_ae)
        print(i)

    filename = '../../02_processed_data/Predictions/all_encoded.sav'
    pickle.dump(all_encoded,     open(filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    filename = '../../02_processed_data/Predictions/AE_labels.sav'
    pickle.dump(all_labels_ae,   open(filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    filename = '../../02_processed_data/Labels/AE_labels.sav'
    pickle.dump(all_labels_ae,   open(filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
filename   = '../../02_processed_data/Labels/AE_labels.sav'
all_aelab  = np.array(pickle.load(open(filename, 'rb')))
filename   = '../../02_processed_data/Labels/SL_labels.sav'
all_stems  = np.array(pickle.load(open(filename, 'rb')))
filename   = '../../02_processed_data/Labels/AO_labels.sav'
avgor      = np.array(pickle.load(open(filename, 'rb')))
a1         = []
a2         = []
a3         = []
a4         = []
a5         = []
a6         = []
a7         = []
a8         = []
for i in range(len(g)):
    
    label1 = np.sum( all_aelab [i] == 0)
    label2 = np.sum( all_aelab [i] == 1)
    label5 = np.sum( all_stems [i] == 0)
    label6 = np.sum( all_stems [i] == 1)
    label7 = np.sum( avgor     [i] == 1)
    label8 = np.sum( avgor     [i] == 0)
    a1.append(label1/obj.totalAtoms)
    a2.append(label2/obj.totalAtoms)
    a5.append(label5/obj.totalAtoms)
    a6.append(label6/obj.totalAtoms)
    a7.append(label7/obj.totalAtoms)
    a8.append(label8/obj.totalAtoms)

a2=np.array(a2)
a6=np.array(a6)
a7=np.array(a7)

np.savetxt('../../03_figures/Fig09/Fig09a.csv', np.concatenate([a2[:,None], a6[:,None], a7[:,None]]).T, delimiter=",")

fig  = plt.figure(figsize=(6,7))
ax   = fig.add_subplot()

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '$t_{pre}$',zorder=-1)
ax.axvline(param["t_tr"] , color = ocol["t_tr"],   ls = '--', lw = 2, label = '$t_{tr}$', zorder=-1)
ax.axvline(param["t_min"], color = ocol["t_min"],  ls = '--', lw = 2, label = '$t_{min}$',zorder=-1)

ax2  = ax.twiny()
ax2.xaxis.tick_top()
tmin = 45
tmax = 76

ax.set_xlim(tmin,tmax)
ax2.set_xlim(control_parameters.calcLJTempAtTime(tmin),control_parameters.calcLJTempAtTime(tmax))

ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11) 
ax2.xaxis.set_label_position('top') 
ax2.tick_params(axis='x', labelsize=19)

limit = len(g)
sf    = 30
f2    = 1.5

ax.scatter(np.arange(limit)+1, a1, s=f2*sf*ps_am, marker = "s", facecolors = 'none', edgecolors = ocol["AE"])
ax.scatter(np.arange(limit)+1, a2, s=sf*ps_cr, c = ocol["AE"],  marker = "s", label = olab["sAE"])
ax.scatter(np.arange(limit)+1, a5, s=f2*sf*ps_am, marker = "v", facecolors = 'none', edgecolors = ocol["SL"])
ax.scatter(np.arange(limit)+1, a6, s=sf*ps_cr, c = ocol["SL"],  marker = "v", label = olab["sSL"])
ax.scatter(np.arange(limit)+1, a8, s=f2*sf*ps_am, marker = "o", facecolors = 'none', edgecolors = ocol["AO"])
ax.scatter(np.arange(limit)+1, a7, s=sf*ps_cr, c = ocol["AO"],  marker = "o", label = olab["sAO"])

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('Crystallinity', fontsize = 20)
ax.set_ylim(0,0.5)
ax.tick_params(axis='both', labelsize = 19)
ax.legend    (fontsize = 20)
ax.grid(alpha = 0.3)
plt.savefig('../../03_figures/Fig09/Fig09a.png',bbox_inches="tight")


In [ ]:
line_wid    = 2
sf          = 3
marker_size = ps_cr*sf
filename    = '../../02_processed_data/Labels/AE_labels.sav'
all_aelab   = np.array(pickle.load(open(filename, 'rb')))
filename    = '../../02_processed_data/Labels/SL_labels.sav'
all_stems   = np.array(pickle.load(open(filename, 'rb')))
filename    = '../../02_processed_data/Labels/AO_labels.sav'
avgor       = np.array(pickle.load(open(filename, 'rb')))

a2          = []
a6          = []
a7          = []

for i in range(len(g)):

    label2 = np.sum( all_aelab [i] == 1)
    label6 = np.sum( all_stems [i] == 1)
    label7 = np.sum( avgor     [i] == 1)
    a2.append(label2)
    a6.append(label6)
    a7.append(label7)

np.savetxt('../../03_figures/Fig10/Fig10a2.csv', np.array([a2, a6, a7]).T, delimiter=",", header="ML labels, SL labels, AO labales")

def error(x):
    return 2*np.sqrt(x)

limit = len(g)
plt.figure(figsize = (8, 5))
plt.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
plt.axvline(param["t_tr"] , color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
plt.axvline(param["t_min"], color = ocol["t_min"],  ls = '--', lw = 2, label = '')

f=1./obj.totalAtoms

plt.errorbar(np.arange(limit)+1, f*np.array(a2), yerr= f*error(np.array(a2)), c=ocol["AE"], lw=1)
plt.plot(np.arange(limit)+1, f*np.array(a2), c = ocol["AE"],   marker="s", lw = line_wid, ms = marker_size)

plt.errorbar(np.arange(limit)+1, f*np.array(a6), yerr= f*error(np.array(a6)), c=ocol["SL"], lw=1)
plt.plot(np.arange(limit)+1, f*np.array(a6), c = ocol["SL"], marker="v", lw = line_wid, ms = marker_size)

plt.errorbar(np.arange(limit)+1, f*np.array(a7), yerr= f*error(np.array(a7)), c=ocol["AO"], lw=1)
plt.plot(np.arange(limit)+1, f*np.array(a7), c = ocol["AO"],  marker="o", lw = line_wid, ms = marker_size)

plt.xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
plt.ylabel('Crystallinity', fontsize = 20)
#plt.legend(fontsize = 23)

plt.yscale('log')
plt.xlim([F10min, F10max])
plt.xticks(fig10_ticks)
#plt.xlim(35, 65)
plt.ylim([f*2, f*6e5])
plt.tight_layout()

plt.savefig('../../03_figures/Fig10/Fig10a2.png',bbox_inches="tight")


In [ ]:
line_wid    = 2
sf          = 3
marker_size = ps_cr*sf
filename    = '../../02_processed_data/Labels/AE_labels.sav'
all_aelab   = np.array(pickle.load(open(filename, 'rb')))
filename    = '../../02_processed_data/Labels/SL_labels2.sav'
all_stems   = np.array(pickle.load(open(filename, 'rb')))
filename    = '../../02_processed_data/Labels/AO_labels2.sav'
avgor       = np.array(pickle.load(open(filename, 'rb')))

a2          = []
a6          = []
a7          = []

for i in range(len(g)):

    label2 = np.sum( all_aelab [i] == 1)
    label6 = np.sum( all_stems [i] == 1)
    label7 = np.sum( avgor     [i] == 1)
    a2.append(label2)
    a6.append(label6)
    a7.append(label7)
    
np.savetxt('../../03_figures/Fig10/Fig10b2.csv', np.array([a2, a6, a7]).T, delimiter=",", header="ML labels, SL labels, AO labales")

def error(x):
    return 2*np.sqrt(x)

limit = len(g)
plt.figure(figsize = (8, 5))
plt.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
plt.axvline(param["t_tr"] , color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
plt.axvline(param["t_min"], color = ocol["t_min"],  ls = '--', lw = 2, label = '')

f=1./obj.totalAtoms

plt.errorbar(np.arange(limit)+1, f*np.array(a2), yerr= f*error(np.array(a2)), c=ocol["AE"], lw=1)
plt.plot(np.arange(limit)+1, f*np.array(a2), c = ocol["AE"],   marker="s", lw = line_wid, ms = marker_size)

plt.errorbar(np.arange(limit)+1, f*np.array(a6), yerr= f*error(np.array(a6)), c=ocol["SL"], lw=1)
plt.plot(np.arange(limit)+1, f*np.array(a6), c = ocol["SL"], marker="v", lw = line_wid, ms = marker_size)

plt.errorbar(np.arange(limit)+1, f*np.array(a7), yerr= f*error(np.array(a7)), c=ocol["AO"], lw=1)
plt.plot(np.arange(limit)+1, f*np.array(a7), c = ocol["AO"],  marker="o", lw = line_wid, ms = marker_size)

plt.xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
plt.ylabel('Crystallinity', fontsize = 20)
plt.tick_params(bottom=True)

plt.yscale('log')
plt.xlim(F10min, F10max)
plt.xticks(fig10_ticks)
plt.ylim([f*2, f*6e5])
plt.tight_layout()

plt.savefig('../../03_figures/Fig10/Fig10b2.png',bbox_inches="tight")


In [ ]:
filename       = '../../02_processed_data/Labels/AE_labels.sav'
all_ae_labels  = np.array(pickle.load(open(filename, 'rb')))
filename       = '../../02_processed_data/Labels/SL_labels.sav'
all_sl_labels  = np.array(pickle.load(open(filename, 'rb')))
filename       = '../../02_processed_data/Labels/AO_labels.sav'
all_ao_labels  = np.array(pickle.load(open(filename, 'rb')))

def error(x, dt):
    n1     = x[:-dt]
    n2     = x[dt: ]
    avg    = (n1 + n2)*0.5
    diff   = n2 - n1
    y      = diff/avg
    fehler = 2*np.sqrt(n1**3+n2**3)/(n1+n2)**2
    return y, 2*fehler

line_wid    = 2
marker_size = 10
dt          = 1
times       = np.arange(0, len(g)-dt, dt)+1
plt.figure(figsize = (8, 5))

s      = np.sum(all_ae_labels == 1, axis = 1)
y, err = error(s, dt)
plt.errorbar(times, y, yerr=err, c=ocol["AE"], lw=1)
plt.plot    (times, y, c = ocol["AE"],   marker="s", lw = line_wid, ms = marker_size, label = 'AE')
data = np.c_[times,  y]

s      = np.sum(all_sl_labels == 1, axis = 1)
y, err = error(s, dt)
plt.errorbar(times, y, yerr=err, c=ocol["SL"], lw=1)
plt.plot    (times, y, c = ocol["SL"], marker="v", lw = line_wid, ms = marker_size, label = 'SL')
data = np.c_[data,  y]

s      = np.sum(all_ao_labels == 1, axis = 1)
y, err = error(s, dt)
plt.errorbar(times, y, yerr=err, c=ocol["AO"], lw=1)
plt.plot    (times, y, c = ocol["AO"],   marker="o", lw = line_wid, ms = marker_size, label = 'AO')
data = np.c_[data,  y]

np.savetxt('../../03_figures/Fig10/Fig10a3.csv', data, delimiter=",")

plt.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, alpha = 0.7)
plt.axvline(param["t_tr"],  color = ocol["t_tr"],   ls = '--', lw = 2, alpha = 0.7)
plt.axvline(param["t_min"], color = ocol["t_min"],  ls = '--', lw = 2, alpha = 0.7)

plt.xlabel('Time, $t$ ($10^6$ MD steps)',fontsize = 20)
plt.ylabel(r'$ \Delta f_{Cryst}(t) $ ', fontsize = 20, labelpad=0)

plt.xlim([F10min, F10max])
plt.xticks(fig10_ticks)
plt.ylim([-1.55, 2.49])
#plt.yscale('log')
plt.savefig('../../03_figures/Fig10/Fig10a3.png',bbox_inches="tight")

In [ ]:
filename       = '../../02_processed_data/Labels/AE_labels.sav'
all_ae_labels  = np.array(pickle.load(open(filename, 'rb')))
filename       = '../../02_processed_data/Labels/SL_labels2.sav'
all_sl_labels  = np.array(pickle.load(open(filename, 'rb')))
filename       = '../../02_processed_data/Labels/AO_labels2.sav'
all_ao_labels  = np.array(pickle.load(open(filename, 'rb')))

def error(x, dt):
    n1     = x[:-dt]
    n2     = x[dt: ]
    avg    = (n1 + n2)*0.5
    diff   = n2 - n1
    y      = diff/avg
    fehler = 2*np.sqrt(n1**3+n2**3)/(n1+n2)**2
    return y, 2*fehler

line_wid    = 2
marker_size = 10
dt          = 1
times       = np.arange(0, len(g)-dt, dt)+1
plt.figure(figsize = (8, 5))

s      = np.sum(all_ae_labels == 1, axis = 1)
y, err = error(s, dt)
plt.errorbar(times, y, yerr=err, c=ocol["AE"], lw=1)
plt.plot    (times, y, c = ocol["AE"],   marker="s", lw = line_wid, ms = marker_size, label = olab["sAE"])
data = np.c_[times,  y]

s      = np.sum(all_sl_labels == 1, axis = 1)
y, err = error(s, dt)
plt.errorbar(times, y, yerr=err, c=ocol["SL"], lw=1)
plt.plot    (times, y, c = ocol["SL"], marker="v", lw = line_wid, ms = marker_size, label = olab["sSLII"])
data = np.c_[data+1,  y]

s      = np.sum(all_ao_labels == 1, axis = 1)
y, err = error(s, dt)
plt.errorbar(times, y, yerr=err, c=ocol["AO"], lw=1)
plt.plot    (times, y, c = ocol["AO"],   marker="o", lw = line_wid, ms = marker_size, label = olab["sAOII"])
data = np.c_[data,  y]

np.savetxt('../../03_figures/Fig10/Fig10b3.csv', data, delimiter=",")

plt.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, alpha = 0.7)
plt.axvline(param["t_tr"],  color = ocol["t_tr"],   ls = '--', lw = 2, alpha = 0.7)
plt.axvline(param["t_min"], color = ocol["t_min"],  ls = '--', lw = 2, alpha = 0.7)
plt.xlabel('Time, $t$ ($10^6$ MD steps)',fontsize = 20)

plt.ylabel(r'$ \Delta f_{Cryst}(t) $ ', fontsize = 20, labelpad=0)
plt.xlim([F10min, F10max])
plt.xticks(fig10_ticks)
plt.ylim([-1.55, 2.49])
plt.savefig('../../03_figures/Fig10/Fig10b3.png',bbox_inches="tight")

## Surviving monomers (number and fraction)

In [ ]:
p_ff      = []
p_ae      = []
p_sl      = []
p_ao      = []
p_ff_frac = []
p_ae_frac = []
p_sl_frac = []
p_ao_frac = []

def calc_Nsurv(labels_array):

    Nsurv = []
    fsurv = []
    
    labels_end = np.where( labels_array[-1]==1 )[0]
    N_c_end    = len(labels_end)
    N_tot      = 1e6
    f_c_end    = N_c_end / N_tot
    
    for t in np.arange(len(labels_array)):
        
        labels_t  = np.where( labels_array[t]==1 )[0]
        survivors = np.in1d(labels_t, labels_end)
        count     = np.sum(survivors)
        n_c       = len(labels_t)
        f         = 0.0
        
        if ( n_c ):
            f = (count/n_c) / f_c_end
                                        
        Nsurv.append(count)
        fsurv.append(f)

    return np.array(Nsurv), np.array(fsurv)

filename        = '../../02_processed_data/Labels/AE_labels.sav'
ae_array        = np.array(pickle.load(open(filename, 'rb')))
p_ae, p_ae_frac = calc_Nsurv(ae_array)

filename        = '../../02_processed_data/Labels/SL_labels.sav'
sl_array        = np.array(pickle.load(open(filename, 'rb')))
p_sl, p_sl_frac = calc_Nsurv(sl_array)

filename        = '../../02_processed_data/Labels/AO_labels.sav'
ao_array        = np.array(pickle.load(open(filename, 'rb')))
p_ao, p_ao_frac = calc_Nsurv(ao_array)

prob      = [p_ae,      p_sl,      p_ao]
prob_frac = [p_ae_frac, p_sl_frac, p_ao_frac]

np.savetxt('../../02_processed_data/Surviving_number.csv',   prob,      delimiter=",")
np.savetxt('../../02_processed_data/Surviving_fraction.csv', prob_frac, delimiter=",")


#### For enhanced boundaries

In [ ]:
p_ff      = []
p_ae      = []
p_sl      = []
p_ao      = []
p_ff_frac = []
p_ae_frac = []
p_sl_frac = []
p_ao_frac = []

filename        = '../../02_processed_data/Labels/AE_labels.sav'
ae_array        = np.array(pickle.load(open(filename, 'rb')))
p_ae, p_ae_frac = calc_Nsurv(ae_array)

filename        = '../../02_processed_data/Labels/SL_labels2.sav'
sl_array        = np.array(pickle.load(open(filename, 'rb')))
p_sl, p_sl_frac = calc_Nsurv(sl_array)

filename        = '../../02_processed_data/Labels/AO_labels2.sav'
ao_array        = np.array(pickle.load(open(filename, 'rb')))
p_ao, p_ao_frac = calc_Nsurv(ao_array)

prob      = [p_ae,      p_sl,      p_ao]
prob_frac = [p_ae_frac, p_sl_frac, p_ao_frac]

np.savetxt('../../02_processed_data/Surviving_number2.csv',   prob,      delimiter=",")
np.savetxt('../../02_processed_data/Surviving_fraction2.csv', prob_frac, delimiter=",")


In [ ]:
filename    = '../../02_processed_data/Surviving_number.csv'
prob        = np.loadtxt(filename, delimiter=",")
redundant   = len(prob[0]) - 40
line_wid    = 1
sf          = 3
marker_size = ps_cr*sf
limit       = len(prob[0])+1
fig         = plt.figure(figsize=(6,7))
ax          = fig.add_subplot()
ax2         = ax.twiny()
temp        = np.linspace(0.9, 0.75, 75)[-redundant:]
ax2.set_xlim(max(temp), min(temp))
ax2.tick_params(axis='x', labelsize=19)
ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11)

np.savetxt('../../03_figures/Fig09/Fig09b.csv', prob, delimiter=",")

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '$t_{pre}$')
ax.axvline(param["t_tr"],  color = ocol["t_tr"],   ls = '--', lw = 2, label = '$t_{tr}$')
ax.axvline(param["t_min"], color = ocol["t_min"],  ls = '--', lw = 2, label = '$t_{min}$')

ax.plot(np.arange(limit)[-redundant:], (prob[0][-redundant:])/1e6, c =ocol["AE"], marker="s", lw = line_wid, ms = marker_size, label = olab["sAE"])
ax.plot(np.arange(limit)[-redundant:], (prob[1][-redundant:])/1e6, c =ocol["SL"], marker="v", lw = line_wid, ms = marker_size, label = olab["sSL"])
ax.plot(np.arange(limit)[-redundant:], (prob[2][-redundant:])/1e6, c =ocol["AO"], marker="o", lw = line_wid, ms = marker_size, label = olab["sAO"])

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('$N_{surv}(t, t_{end})~/~N_{tot} $', fontsize = 20)

ax.tick_params(axis='both', labelsize=19)
tmin = 45
tmax = 76
ax.set_xlim(tmin,tmax)
ax2.set_xlim(control_parameters.calcLJTempAtTime(tmin), control_parameters.calcLJTempAtTime(tmax))
ax.grid(alpha = 0.3)
plt.savefig('../../03_figures/Fig09/Fig09b.png',bbox_inches="tight")


In [ ]:
tlabel_y = 2.0
ymin     = -0.1
ymax     = 2.2

In [ ]:
file_num    = -1
obj         = ReadCoordinates(g[file_num], param["chainlength"])

filename    = '../../02_processed_data/Surviving_number.csv'
Nsurv       = np.loadtxt(filename, delimiter=",")
filename    = '../../02_processed_data/Surviving_fraction.csv'
prob        = np.loadtxt(filename, delimiter=",")

line_wid    = 2
sf          = 3
marker_size = ps_cr*sf
limit       = len(prob[0])
fig         = plt.figure(figsize=(9,7))
ax          = fig.add_subplot()
temp        = np.linspace(0.9, 0.75, 75)[-redundant:]

ax2         = ax.twiny()

ax2.set_xticks(ticks=np.arange(0.9,   0.75, -0.01))
print ( "ticks at , ", np.arange(0.9, 0.75, -0.05))
ax2.set_xlim(control_parameters.calcLJTempAtTime(F10min), control_parameters.calcLJTempAtTime(F10max))
ax2.tick_params(axis='x', labelsize=19)

ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11)

np.savetxt('../../03_figures/Fig10/Fig10a1.csv', prob, delimiter=",")

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
ax.axvline(param["t_tr"],  color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
ax.axvline(param["t_min"], color = ocol["t_min"],  ls = '--', lw = 2, label = '')

ax.text(param["t_pre"]+.5,tlabel_y, '$t_{pre}$', color = ocol["t_pre"], fontsize=25)
ax.text(param["t_tr"] +.5,tlabel_y, '$t_{tr}$',  color = ocol["t_tr"] , fontsize=25)
ax.text(param["t_min"]+.5,tlabel_y, '$t_{min}$', color = ocol["t_min"], fontsize=25)

x   = np.arange(limit)+1

num = 0
t_y = prob[num]
y   = t_y
err = np.sqrt(Nsurv[num])/Nsurv[num]
ax.plot    (x, y, c = ocol["AE"],   marker="s", lw = line_wid, ms = marker_size, label = olab["sAE"])
ax.errorbar(x, y, yerr=err*y, c = ocol["AE"], ls="--", lw = 0.7, capsize=3, capthick=0.5)

num = 1
t_y = prob[num]
y   = t_y
err = np.sqrt(Nsurv[num])/Nsurv[num]
ax.plot    (x, y, c = ocol["SL"], marker="v", lw = line_wid, ms = marker_size, label = olab["sSL"])
ax.errorbar(x, y, yerr=err*y, c = ocol["SL"], ls="--", lw = 0.7, capsize=3, capthick=0.5)

num = 2
t_y = prob[num]
y   = t_y
err = np.sqrt(Nsurv[num])/Nsurv[num]
ax.plot    (x, y, c = ocol["AO"],  marker="o", lw = line_wid, ms = marker_size, label = olab["sAO"])
ax.errorbar(x, y, yerr=err*y, c = ocol["AO"], ls="--", lw = 0.7, capsize=3, capthick=0.6)

#ax.set_xticks(ticks=np.arange(0, 100, 5))
ax.set_xlim(F10min, F10max)
ax.set_xticks(fig10_ticks)
ax.set_ylim(ymin, ymax)
ax.set_yticks(ticks=np.arange(0,2.01,0.5))

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('$f_{surv} (t, t_{end})$', fontsize = 25)

ax.tick_params(axis='both', labelsize=20)
ax.legend(fontsize = 25, loc = 'lower right')
plt.tight_layout()

plt.savefig('../../03_figures/Fig10/Fig10a1.png',bbox_inches="tight")

In [ ]:
file_num    = -1
obj         = ReadCoordinates(g[file_num], param["chainlength"])

filename    = '../../02_processed_data/Surviving_number2.csv'
Nsurv       = np.loadtxt(filename, delimiter=",")
filename    = '../../02_processed_data/Surviving_fraction2.csv'
prob        = np.loadtxt(filename, delimiter=",")

line_wid    = 2
sf          = 3
marker_size = ps_cr*sf
limit       = len(prob[0])
fig         = plt.figure(figsize=(9,7))
ax          = fig.add_subplot()
ax2         = ax.twiny()

ax2.set_xticks(ticks=np.arange(0.9,   0.75, -0.01))
ax2.set_xlim(control_parameters.calcLJTempAtTime(F10min),control_parameters.calcLJTempAtTime(F10max))
ax2.tick_params(axis='x', labelsize=19)

ax.set_xticks(ticks=np.arange(0,100,5))
ax2.set_xlabel('Temperature (LJ-units)', fontsize = 20, labelpad = 11)

np.savetxt('../../03_figures/Fig10/Fig10b1.csv', prob, delimiter=",")

ax.axvline(param["t_pre"], color = ocol["t_pre"],  ls = '--', lw = 2, label = '')
ax.axvline(param["t_tr"],  color = ocol["t_tr"],   ls = '--', lw = 2, label = '')
ax.axvline(param["t_min"], color = ocol["t_min"],  ls = '--', lw = 2, label = '')

ax.text(param["t_pre"]+.5, tlabel_y,'$t_{pre}$', color = ocol["t_pre"], fontsize=25)
ax.text(param["t_tr"] +.5, tlabel_y,'$t_{tr}$',  color = ocol["t_tr"] , fontsize=25)
ax.text(param["t_min"]+.5, tlabel_y,'$t_{min}$', color = ocol["t_min"], fontsize=25)

x   = np.arange(limit)+1

num = 0
t_y = prob[num]
y   = t_y
err = np.sqrt(Nsurv[num])/Nsurv[num]
ax.plot    (x, y, c = ocol["AE"],   marker="s", lw = line_wid, ms = marker_size, label = olab["sAE"])
ax.errorbar(x, y, yerr=err*y, c = ocol["AE"], ls="--", lw = 0.7, capsize=3, capthick=0.5)

num = 1
t_y = prob[num]
y   = t_y
err = np.sqrt(Nsurv[num])/Nsurv[num]
ax.plot    (x, y, c = ocol["SL"], marker="v", lw = line_wid, ms = marker_size, label = olab["sSLII"])
ax.errorbar(x, y, yerr=err*y, c = ocol["SL"], ls="--", lw = 0.7, capsize=3, capthick=0.5)

num = 2
t_y = prob[num]
y   = t_y
err = np.sqrt(Nsurv[num])/Nsurv[num]
ax.plot    (x, y, c = ocol["AO"],  marker="o", lw = line_wid, ms = marker_size, label = olab["sAOII"])
ax.errorbar(x, y, yerr=err*y, c = ocol["AO"], ls="--", lw = 0.7, capsize=3, capthick=0.6)

ax.set_xlim(F10min, F10max)
ax.set_xticks(fig10_ticks)
ax.set_ylim(ymin, ymax)

ax.set_xlabel('Time, $t$ ($10^6$ MD steps)', fontsize = 20)
ax.set_ylabel('$f_{surv} (t, t_{end})$', fontsize = 25)

ax.tick_params(axis='both', labelsize=20)
ax.legend(fontsize = 25, loc = 'lower right')
plt.tight_layout()

plt.savefig('../../03_figures/Fig10/Fig10b1.png',bbox_inches="tight")

### Classification based on the average orientation

In [ ]:
file_num       = -1
filename       = '../../02_processed_data/Training_data/ae_training_idxs.csv'
rnd            = np.int64(np.loadtxt(filename, delimiter=","))
filename       = '../../02_processed_data/Labels/AO_labels.sav'
avgor          = np.array(pickle.load(open(filename, 'rb')))[file_num]
target_resc    = avgor[rnd]
filename       = '../../02_processed_data/Predictions/all_encoded.sav'
encoded        = np.array(pickle.load(open(filename, 'rb')))[file_num][rnd]

np.savetxt('../../03_figures/FigS6/FigS6b.csv', np.c_[encoded, target_resc], delimiter=",")

class1_idx     = np.where(target_resc == 1)[0]
class0_idx     = np.where(target_resc == 0)[0]

fig      = plt.figure()
p1       = encoded[class0_idx]
plt.scatter(p1[:,1], p1[:,0], c = ocol["amorph"],  s = ps_am, label = olab["sAO"]+olab["am"])
p1       = encoded[class1_idx]
plt.scatter(p1[:,1], p1[:,0], c = ocol["AO"], s = ps_cr, label = olab["sAO"]+olab["cr"])

our_colors.formatMap(plt)

plt.savefig('../../03_figures/FigS6/FigS6b.png',bbox_inches="tight")


In [ ]:
rnd           = get_slice_indices(obj, 100, 50, axis = 2)
target_resc   = avgor[rnd]
class1_idx    = np.where(target_resc == 1)[0]
class0_idx    = np.where(target_resc == 0)[0]
selected      = obj.folded[rnd]

np.savetxt('../../03_figures/FigS7/FigS7b.csv', np.c_[selected[:, :2], target_resc], delimiter=",")

fig      = plt.figure()
p1       = selected[class0_idx]
plt.scatter(p1[:,0], p1[:,1], c = ocol["amorph"], s = ps_am, label =  olab["sAO"]+olab["am"])
p1       = selected[class1_idx]
plt.scatter(p1[:,0], p1[:,1], c = ocol["AO"], s = ps_cr, label =  olab["sAO"]+olab["cr"])
lgnd     = plt.legend(fontsize=13, loc = 'best')
for handle in lgnd.legendHandles:
    handle.set_sizes([40.0])

plt.xlabel('X $(\sigma)$', fontsize = 20)
plt.ylabel('Y $(\sigma)$', fontsize = 20)
plt.tight_layout()
plt.savefig('../../03_figures/FigS7/FigS7b.png',bbox_inches="tight")


### Classification based on the stem length

In [ ]:
file_num       = -1
filename       = '../../02_processed_data/Training_data/ae_training_idxs.csv'
rnd            = np.int64(np.loadtxt(filename, delimiter=","))
filename       = '../../02_processed_data/Labels/SL_labels.sav'
all_stems      = np.array(pickle.load(open(filename, 'rb')))[file_num][rnd]
filename       = '../../02_processed_data/Predictions/all_encoded.sav'
encoded        = np.array(pickle.load(open(filename, 'rb')))[file_num][rnd]

np.savetxt('../../03_figures/FigS6/FigS6a.csv', np.c_[encoded, all_stems], delimiter=",")

fig = plt.figure()
p1  = encoded[np.where(all_stems == 0)[0]]
plt.scatter(p1[:,1], p1[:,0], c = ocol["amorph"],  s = ps_am, label = olab["sSL"]+olab["am"])
p1  = encoded[np.where(all_stems == 1)[0]]
plt.scatter(p1[:,1], p1[:,0], c = ocol["SL"]*0.8,  s = ps_cr, label = olab["sSL"]+olab["cr"])

our_colors.formatMap(plt)

plt.savefig('../../03_figures/FigS6/FigS6a.png',bbox_inches="tight")


In [ ]:
file_num    = -1
obj         = ReadCoordinates(g[file_num], param["chainlength"])
rnd         = get_slice_indices(obj, 100, 50, axis = 2)
selected    = obj.folded[rnd]
filename    = '../../02_processed_data/Labels/SL_labels.sav'
all_stems   = np.array(pickle.load(open(filename, 'rb')))[file_num][rnd]

np.savetxt('../../03_figures/FigS7/FigS7a.csv', np.c_[selected[:, :2], all_stems], delimiter=",")

fig = plt.figure()
p1  = selected[np.where(all_stems == 0)[0]]
plt.scatter(p1[:,0], p1[:,1], c = ocol["amorph"],  s = ps_am, label = olab["sSL"]+olab["am"])
p1  = selected[np.where(all_stems == 1)[0]]
plt.scatter(p1[:,0], p1[:,1], c = ocol["SL"]*0.8, s = ps_cr, label = olab["sSL"]+olab["cr"])

lgnd     = plt.legend(fontsize=13)
for handle in lgnd.legendHandles:
    handle.set_sizes([40.0])

plt.xlabel('X $(\sigma)$', fontsize = 20)
plt.ylabel('Y $(\sigma)$', fontsize = 20)
plt.tight_layout()
plt.savefig('../../03_figures/FigS7/FigS7a.png',bbox_inches="tight")

## Comparisons on the cross section of the simulation box

### 3d box with slices highlighted

In [ ]:
timestep = -1
obj      = ReadCoordinates(g[timestep], param["chainlength"])
filename = '../../02_processed_data/Labels/AE_labels.sav'
ae_lab   = np.array(pickle.load(open(filename, 'rb')))[timestep]


In [ ]:
fig = plt.figure(figsize=(6,6))
ax  = fig.add_subplot(111, projection = '3d')

xlo = obj.boxMin[0] 
xhi = obj.boxMax[0]
ylo = obj.boxMin[1]
yhi = obj.boxMax[1]
zlo = obj.boxMin[2]
zhi = obj.boxMax[2]
col = ['red', 'blue']

rnd    = get_slice_indices(obj, 100, 15, 2)
lab    = ae_lab[rnd]
p1     = obj.folded[rnd][lab == 0]
ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.08, c = ocol["amorph"], label = olab["sAE"]+olab["am"])
p1     = obj.folded[rnd][lab == 1]
ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.15, c = ocol["AE"]*0.8, label = olab["sAE"]+olab["cr"])
ax.text(0, 0, 15, '15th slice', c = 'black', fontsize = 15)
box1   = np.c_[obj.folded[rnd], lab]

rnd    = get_slice_indices(obj, 100, 50, 2)
lab    = ae_lab[rnd]
p1     = obj.folded[rnd][lab == 0]
ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.08, c =  ocol["amorph"])
p1     = obj.folded[rnd][lab == 1]
ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.15, c =  ocol["AE"]*0.8)
ax.text(0, 0, 50, '50th slice', c = 'black', fontsize = 15)
box2   = np.c_[obj.folded[rnd], lab]

id1    = np.full((box1.shape[0], 1), 'slice15')
id2    = np.full((box2.shape[0], 1), 'slice50')
array1 = np.hstack((id1, box1))
array2 = np.hstack((id2, box2))
comb   = np.vstack((array1, array2))
cols   = ['slice#', 'x', 'y', 'z', 'AE']
df     = pd.DataFrame(comb, columns=cols)
df.to_csv('../../03_figures/Fig08/Fig08a.csv', index=False)

r      = [obj.boxMin[0], obj.boxMax[0] ]
for s, e in combinations(np.array(list(product(r, r, r))), 2):
    if np.sum(np.abs(s-e)) == r[1]-r[0]:
        ax.plot3D(*zip(s, e), color=ocol["SL"])

ax.view_init(13, 10)

lgnd     = plt.legend(fontsize=13, loc = "best")
for handle in lgnd.legendHandles:
        handle.set_sizes([40.0])
ax.set_xlim3d(xlo-3, xhi+3)
ax.set_ylim3d(ylo-3, yhi+3)
ax.set_zlim3d(zlo-3, zhi+3)
ax.set_xlabel('X $(\sigma)$', fontsize = 15, labelpad = 11)
ax.set_ylabel('Y $(\sigma)$', fontsize = 15, labelpad = 11)
ax.set_zlabel('Z $(\sigma)$', fontsize = 15)
plt.savefig('../../03_figures/Fig08/Fig08a.png',bbox_inches="tight")


### ML cross section

In [ ]:
rnd            = get_slice_indices(obj, 100, 50, axis = 2)
selected       = obj.folded[rnd]
filename       = '../../02_processed_data/Labels/AE_labels.sav'
ae_lab         = np.array(pickle.load(open(filename, 'rb')))[file_num][rnd]
filename       = '../../02_processed_data/Labels/AO_labels.sav'
ao_lab         = np.array(pickle.load(open(filename, 'rb')))[file_num][rnd]

In [ ]:
np.savetxt('../../03_figures/Fig08/Fig08c.csv', np.c_[selected[:, :2], ae_lab, ao_lab], delimiter=",")

fig      = plt.figure()

p1       = selected[ao_lab==1]
plt.scatter(p1[:,0], p1[:,1], facecolors = 'none', edgecolors = ocol["AO"]*1., lw = .9, s=6.7,alpha = 1, label = olab["sAO"]+olab["cr"])    

p1       = selected[ae_lab==1]
plt.scatter(p1[:,0], p1[:,1], c = ocol["AE"]*0.8,  s = 1.3,  alpha = 1., label = olab["sAE"]+olab["cr"])

p1       = selected[ae_lab==0]
plt.scatter(p1[:,0], p1[:,1], c = ocol["amorph"],  s = 1,  alpha = 0.6, label = olab["sAE"]+olab["am"])

plt.xlabel('X', fontsize = 20)
plt.ylabel('Y', fontsize = 20)
lgnd     = plt.legend(fontsize = 13, loc = 'best')
for handle in lgnd.legendHandles:
    handle.set_sizes([40.0])
plt.tight_layout()

plt.savefig('../../03_figures/Fig08/Fig08c.png',bbox_inches="tight")


In [ ]:
selected    = obj.folded[rnd]
filename    = '../../02_processed_data/Labels/AE_labels.sav'
ae_lab      = np.array(pickle.load(open(filename, 'rb')))[file_num][rnd]
filename    = '../../02_processed_data/Labels/SL_labels.sav'
all_stems   = np.array(pickle.load(open(filename, 'rb')))[file_num][rnd]

In [ ]:
np.savetxt('../../03_figures/Fig08/Fig08d.csv', np.c_[selected[:, :2], ae_lab, all_stems], delimiter=",")

fig      = plt.figure()

p1       = selected[np.where(all_stems == 1)[0]]
plt.scatter(p1[:,0], p1[:,1], facecolors = 'none', edgecolors = ocol["SL"]*.8, lw = .9, s=6.7,alpha = 1, label = olab["sSL"]+olab["cr"])

p1       = selected[ae_lab==1]
plt.scatter(p1[:,0], p1[:,1], c = ocol["AE"]*0.8,  s = 1.3,  alpha = 1., label = olab["sAE"]+olab["cr"])

p1       = selected[ae_lab==0]
plt.scatter(p1[:,0], p1[:,1], c = ocol["amorph"],   s = 1,   alpha = 0.5, label = olab["sAE"]+olab["am"])
    
plt.xlabel('X', fontsize = 20)
plt.ylabel('Y', fontsize = 20)
lgnd     = plt.legend(fontsize = 13, loc = 'best')
for handle in lgnd.legendHandles:
    handle.set_sizes([40.0])
plt.tight_layout()
plt.savefig('../../03_figures/Fig08/Fig08d.png',bbox_inches="tight")
